In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from PIL import Image
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive/Cats and Dogs/Dataset"
start = time.perf_counter()

In [4]:
from torchvision.transforms.transforms import Normalize
# pre = process data
transform = transforms.Compose([transforms.Resize((32,32)),
            transforms.ToTensor(),
            transforms.transforms.Normalize([0.5],[0.5])
])
dataset = datasets.ImageFolder('/content/drive/MyDrive/Cats and Dogs/Dataset', transform = transform)
dataset_len = len(dataset)
train_len, test_len = dataset_len-6000, 6000
train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])
batch_size = 200
train_set = DataLoader(dataset = train_set, shuffle=True, batch_size = batch_size)
test_set = DataLoader (dataset = test_set, shuffle=True, batch_size = batch_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(train_len,test_len)

2025 6000


In [5]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Model().to(device)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr = 0.001, weight_decay = 1e-5)


In [7]:
net.train()
for epoch in range(50):
  total_correct = 0.0
  running_loss = 0.0
  for i, (inputs, labels) in enumerate(train_set):
     inputs, labels = inputs.to(device), labels.to(device)
     output = net(inputs)
     output_idx = torch.argmax(output, dim=1)
     total_correct += (labels == output_idx).sum().item()
     optimizer.zero_grad()
     loss = criterion(output, labels)
     loss.backward()
     optimizer.step()
  print(f'Epoch: {epoch+1} Loss: {loss.item()} accuracy = {100*(total_correct/train_len)}%')

print('Finished training')
end = time.perf_counter()
time_train = end - start
print(f"time taken for training running in GPU {time_train:.4f} sec")


Epoch: 1 Loss: 1.357487440109253 accuracy = 47.358024691358025%
Epoch: 2 Loss: 0.7866372466087341 accuracy = 50.123456790123456%
Epoch: 3 Loss: 0.700207531452179 accuracy = 50.864197530864196%
Epoch: 4 Loss: 0.6849862933158875 accuracy = 51.65432098765432%
Epoch: 5 Loss: 0.6811452507972717 accuracy = 53.629629629629626%
Epoch: 6 Loss: 0.6980013251304626 accuracy = 51.456790123456784%
Epoch: 7 Loss: 0.6770684719085693 accuracy = 52.54320987654321%
Epoch: 8 Loss: 0.6754978895187378 accuracy = 54.32098765432099%
Epoch: 9 Loss: 0.7186625003814697 accuracy = 55.30864197530864%
Epoch: 10 Loss: 0.6265102624893188 accuracy = 56.395061728395056%
Epoch: 11 Loss: 0.6517246961593628 accuracy = 54.666666666666664%
Epoch: 12 Loss: 0.6666306257247925 accuracy = 57.18518518518518%
Epoch: 13 Loss: 0.72466641664505 accuracy = 58.51851851851851%
Epoch: 14 Loss: 0.6779674291610718 accuracy = 54.91358024691358%
Epoch: 15 Loss: 0.7221893072128296 accuracy = 52.79012345679013%
Epoch: 16 Loss: 0.6686246395111

In [ ]:
s = time.perf_counter()
with torch.no_grad():
  total_loss = 0.0
  total_correct = 0.0
  for inputs, labels in test_set:
    labels = labels.to(device)
    outputs = net(inputs.to(device))
    loss = criterion(outputs, labels)
    output_idx = torch.argmax(outputs, dim=1)
    total_correct += sum(labels==output_idx)
  print(f'Accuracy : {100*(total_correct/test_len)} Loss: {loss.item()}')
  e = time.perf_counter()
time_test = e - s
print(f"time taken for test set running in GPU {time_test:.4f} sec")


In [ ]:
!nvidia-smi

Thu Oct 20 11:09:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    25W /  70W |   1158MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------